In [1]:
class NodeTree(object):
    def __init__(self):
        self.left = None
        self.right = None
        self.parent = None


class BinaryTree(object):
    def __init__(self):
        super(BinaryTree, self).__init__()
        self.nil = None
        self.root = None
        self.initialize()

    def initialize(self):
        self.root = None

    def node_depth(self, node: NodeTree):
        """
        сколько нод надо пройти до рута
        """
        tree_depth = 0
        while node != self.root:
            node = node.parent
            tree_depth += 1
        return tree_depth

    def node_size(self, node: NodeTree):
        """
        количество нод расположенных ниже,
        включая переданную ноду
        """
        if node == self.nil:
            return 0
        else:
            left_size = self.node_size(node.left)
            right_size = self.node_size(node.right)
            return 1 + right_size + left_size

    def node_height(self, node: NodeTree):
        if node == self.nil:
            return 0
        else:
            left_size = self.node_size(node.left)
            right_size = self.node_size(node.right)
            return 1 + max(right_size, left_size)

    def traverse_recursive(self, node: NodeTree):
        if node != self.nil:
            self.traverse_recursive(node.left)
            self.traverse_recursive(node.right)

    def traverse_iterative(self):
        node: NodeTree = self.root
        prev_node: NodeTree = self.nil
        next_node: NodeTree = self.nil

        while node != self.nil:
            # если прошлая нода не является родителем
            if prev_node == node.parent:
                # если нода слева не равна null тогда
                # следующая нода будет left
                if node.left != self.nil:
                    next_node = node.left
                # иначе правая
                elif node.right != self.nil:
                    next_node = node.right
                else:
                    # если нода не имеет потомков, тогда
                    # следующей нодой будет родитель
                    next_node = node.parent
            # если предыдущая нода это левая текущей
            elif prev_node == node.left:
                # если правая текущей не пусто,
                # значит она будет следующая
                if node.right != self.nil:
                    next_node = node.right
                # если пустая, значит следующая будет родителем
                else:
                    next_node = node.parent
            else:
                # если все
                next_node = node.parent

            prev_node = node
            node = next_node

In [3]:
"""Some base classes inherited by others"""


class BaseCollection(object):
    """Base class for everything"""

    def __init__(self):
        super(BaseCollection, self).__init__()

    def size(self):
        """This implementation works for almost every class in ODS"""
        return self.n

    def __len__(self):
        return self.size()

    def __str__(self):
        return "[" + ",".join([str(x) for x in self]) + "]"

    def __repr__(self):
        return self.__class__.__name__ + "([" + ",".join([repr(x) for x in self]) + "])"


class BaseSet(BaseCollection):
    """Base class for Set implementations"""

    def __init__(self):
        super(BaseSet, self).__init__()

    def add_all(self, a):
        for x in a:
            self.add(x)

    def __in__(self, x):
        return self.find(x) is not None

    def __eq__(self, a):
        if len(a) != len(self):
            return False
        for x in self:
            if not x in a:
                return False
        for x in a:
            if not x in self:
                return False
        return True

    def __ne__(self, a):
        return not self == a


class BaseList(BaseCollection):
    """Base class for List implementations"""

    def __init__(self):
        super(BaseList, self).__init__()

    def append(self, x):
        self.add(self.size(), x)

    def add_all(self, iterable):
        for x in iterable:
            self.append(x)

    def clear(self):
        """This can be overridden with more efficient implementations"""
        while self.size() > 0:
            self.remove(self.size() - 1)

    def add_first(self, x):
        return self.add(0, x)

    def remove_first(self):
        return self.remove(0)

    def add_last(self, x):
        return self.add(self.size(), x)

    def remove_last(self):
        return self.remove(self.size() - 1)

    def insert(i, x):
        self.add(i, x)

    def __iter__(self):
        """This implementation is good enough for array-based lists"""
        for i in range(len(self)):
            yield (self.get(i))

    def __eq__(self, a):
        if len(a) != len(self):
            return False
        it1 = iter(a)
        it2 = iter(self)
        for i in range(len(a)):
            if it1.next() != it2.next():
                return False
        return True

    def __ne__(self, a):
        return not self == a

    def index(self, x):
        i = 0
        for y in self:
            if y == x:
                return i
            i += 1
        raise ValueError("%r is not in the list" % x)

    def remove_value(self, x):
        try:
            return self.remove(self.index(x))
        except ValueError:
            return False

    def __getitem__(self, key):
        return self.get(key)

    def __setitem__(self, key, value):
        return self.set(key, value)

    def __delitem__(self, i):
        self.remove(i)


import numpy

w = 32


def new_array(n, dtype=object):
    return numpy.empty(n, dtype)


def _new_array(n):
    return [None] * n


def new_zero_array(n):
    return numpy.zeros(n)


def new_boolean_matrix(n, m):
    return numpy.zeros([n, n], numpy.bool_)


def new_boolean_array(n):
    return numpy.zeros(n, numpy.bool_)


def new_int_array(n, init=0):
    a = numpy.empty(n, numpy.int32)
    a.fill(init)
    return a


def binfmt(n):
    return "{0:012b}".format(n)


class ArrayQueue(BaseSet):
    def __init__(self, iterable=[]):
        self._initialize()
        self.add_all(iterable)

    def _initialize(self):
        self.a = new_array(1)
        self.j = 0
        self.n = 0

    def _resize(self):
        b = new_array(max(1, 2 * self.n))
        for k in range(self.n):
            b[k] = self.a[(self.j + k) % len(self.a)]
        self.a = b
        self.j = 0

    def add(self, x):
        if self.n + 1 > len(self.a):
            self._resize()
        self.a[(self.j + self.n) % len(self.a)] = x
        self.n += 1
        return True

    def remove(self):
        if self.n == 0:
            raise IndexError()
        x = self.a[self.j]
        self.j = (self.j + 1) % len(self.a)
        self.n -= 1
        if len(self.a) >= 3 * self.n:
            self._resize()
        return x


class BinaryTree(object):
    class Node(object):
        def __init__(self):
            self.left = self.right = self.parent = None

    def __init__(self):
        super(BinaryTree, self).__init__()
        self.nil = None
        self.r = None
        self.initialize()

    # Initialize function
    def initialize(self):
        self.r = None

    # Function to find depth
    def depth(self, u):
        d = 0
        while u != self.r:
            u = u.parent
            d += 1
        return d

    # Function to find size
    def size(self):
        return self._size(self.r)

    # Helper function for size
    def _size(self, u):
        if u == self.nil:
            return 0
        return 1 + self._size(u.left) + self._size(u.right)

    # Function to find size of the tree
    def size2(self):
        u = self.r
        prv = self.nil
        n = 0
        while u != self.nil:
            if prv == u.parent:
                n += 1
                if u.left != self.nil:
                    nxt = u.left
                elif u.right != self.nil:
                    nxt = u.right
                else:
                    nxt = u.parent
            elif prv == u.left:
                if u.right != self.nil:
                    nxt = u.right
                else:
                    nxt = u.parent
            else:
                nxt = u.parent
            prv = u
            u = nxt
        return n

    # Function to find height
    def height(self):
        return self._height(self.r)

    # Helper function for height
    def _height(self, u):
        if u == self.nil:
            return 0
        return 1 + max(self._height(u.left), self._height(u.right))

    # Function to traverse left and right side of tree
    def traverse(self, u):
        if u == self.nil:
            return
        self.traverse(u.left)
        self.traverse(u.right)

    # Function to traverse the tree
    def traverse2(self):
        u = self.r
        prv = self.nil
        while u != self.nil:
            if prv == u.parent:
                if u.left != self.nil:
                    nxt = u.left
                elif u.right != self.nil:
                    nxt = u.right
                else:
                    nxt = u.parent
            elif prv == u.left:
                if u.right != self.nil:
                    nxt = u.right
                else:
                    nxt = u.parent
            else:
                nxt = u.parent
            prv = u
            u = nxt

    # Function for bf traverse
    def bf_traverse(self):
        q = ArrayQueue()
        if self.r != self.nil:
            q.add(self.r)
        while q.size() > 0:
            u = q.remove()
            if u.left != self.nil:
                q.add(u.left)
            if u.right != self.nil:
                q.add(u.right)

    # Function to find first node
    def first_node(self):
        w = self.r
        if w == self.nil:
            return self.nil
        while w.left != self.nil:
            w = w.left
        return w

    # Function to find next node
    def next_node(self, w):
        if w.right != self.nil:
            w = w.right
            while w.left != self.nil:
                w = w.left
        else:
            while w.parent != self.nil and w.parent.left != w:
                w = w.parent
            w = w.parent
        return w


if __name__ == "__main__":
    # Instantiate a BinaryTree object
    bt = BinaryTree()
    # Create nodes and set up the tree structure
    nilnode = bt.Node()
    node1 = bt.Node()
    node2 = bt.Node()
    node3 = bt.Node()
    node1.parent = nilnode
    node1.left = node2
    node1.right = node3
    node2.parent = node3.parent = node1
    # Initialize the BinaryTree object
    bt.r = node1
    # Print the results of various BinaryTree methods
    print("Depth:", bt.depth(node2))
    print("Size:", bt.size())
    print("Size2:", bt.size2())
    print("Height:", bt.height())

Depth: 1
Size: 3
Size2: 0
Height: 2
